In [ ]:
# robot_web_interface.ipynb
# Jupyter notebook for robot control and analysis

import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
import websocket
import json
import threading
import time

class RobotWebInterface:
    def __init__(self, robot_ip):
        self.robot_ip = robot_ip
        self.websocket_url = f"ws://{robot_ip}:9090"
        self.web_interface_url = f"http://{robot_ip}:5000"
        self.data_buffer = []
        self.ws = None
        
    def create_control_interface(self):
        """Create interactive control interface in Jupyter"""
        control_html = f"""
        <div style="border: 2px solid #4CAF50; padding: 20px; border-radius: 10px; background: #f9f9f9;">
            <h3>Robot Control Interface</h3>
            <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px; max-width: 300px;">
                <div></div>
                <button onclick="sendCommand('forward')" style="padding: 15px; font-size: 14px;">↑ Forward</button>
                <div></div>
                
                <button onclick="sendCommand('left')" style="padding: 15px; font-size: 14px;">← Left</button>
                <button onclick="sendCommand('stop')" style="padding: 15px; font-size: 14px; background: red; color: white;">STOP</button>
                <button onclick="sendCommand('right')" style="padding: 15px; font-size: 14px;">→ Right</button>
                
                <div></div>
                <button onclick="sendCommand('backward')" style="padding: 15px; font-size: 14px;">↓ Backward</button>
                <div></div>
            </div>
            
            <div style="margin-top: 20px;">
                <label>Speed: <input type="range" id="speed-slider" min="0.1" max="1.0" step="0.1" value="0.5"></label>
                <span id="speed-value">0.5</span> m/s
            </div>
            
            <div style="margin-top: 20px;">
                <button onclick="startDataLogging()" style="padding: 10px; background: green; color: white;">Start Logging</button>
                <button onclick="stopDataLogging()" style="padding: 10px; background: orange; color: white;">Stop Logging</button>
                <button onclick="downloadData()" style="padding: 10px; background: blue; color: white;">Download Data</button>
            </div>
            
            <div id="status" style="margin-top: 15px; padding: 10px; background: #e7e7e7;">
                Status: <span id="connection-status">Disconnected</span>
            </div>
        </div>
        
        <script>
        var ros_connection = new WebSocket('{self.websocket_url}');
        var logging_active = false;
        
        ros_connection.onopen = function() {{
            document.getElementById('connection-status').textContent = 'Connected';
            document.getElementById('connection-status').style.color = 'green';
        }};
        
        function sendCommand(direction) {{
            var speed = parseFloat(document.getElementById('speed-slider').value);
            var cmd = {{}};
            
            switch(direction) {{
                case 'forward': cmd = {{linear: {{x: speed}}, angular: {{z: 0}}}}; break;
                case 'backward': cmd = {{linear: {{x: -speed}}, angular: {{z: 0}}}}; break;
                case 'left': cmd = {{linear: {{x: 0}}, angular: {{z: speed}}}}; break;
                case 'right': cmd = {{linear: {{x: 0}}, angular: {{z: -speed}}}}; break;
                case 'stop': cmd = {{linear: {{x: 0}}, angular: {{z: 0}}}}; break;
            }}
            
            var message = {{
                op: 'publish',
                topic: '/cmd_vel',
                type: 'geometry_msgs/Twist',
                msg: cmd
            }};
            
            ros_connection.send(JSON.stringify(message));
        }}
        
        document.getElementById('speed-slider').oninput = function() {{
            document.getElementById('speed-value').textContent = this.value;
        }};
        </script>
        """
        display(HTML(control_html))
    
    def load_data_from_csv(self, filename):
        """Load robot data from CSV file"""
        try:
            data = pd.read_csv(filename)
            return data
        except FileNotFoundError:
            print(f"File {filename} not found. Make sure to export data from web interface first.")
            return None
    
    def analyze_robot_performance(self, data):
        """Analyze robot performance data"""
        if data is None:
            return
            
        # Create analysis plots
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # Plot trajectory
        axes[0,0].plot(data['x_position'], data['y_position'], 'b-', linewidth=2)
        axes[0,0].set_xlabel('X Position (m)')
        axes[0,0].set_ylabel('Y Position (m)')
        axes[0,0].set_title('Robot Trajectory')
        axes[0,0].grid(True)
        axes[0,0].axis('equal')
        
        # Plot velocity over time
        axes[0,1].plot(data['time_elapsed'], data['velocity'], 'r-', linewidth=1.5)
        axes[0,1].set_xlabel('Time (s)')
        axes[0,1].set_ylabel('Velocity (m/s)')
        axes[0,1].set_title('Velocity Profile')
        axes[0,1].grid(True)
        
        # Plot position components
        axes[1,0].plot(data['time_elapsed'], data['x_position'], 'r-', label='X Position')
        axes[1,0].plot(data['time_elapsed'], data['y_position'], 'b-', label='Y Position')
        axes[1,0].set_xlabel('Time (s)')
        axes[1,0].set_ylabel('Position (m)')
        axes[1,0].set_title('Position vs Time')
        axes[1,0].legend()
        axes[1,0].grid(True)
        
        # Performance statistics
        total_distance = np.sum(np.sqrt(np.diff(data['x_position'])**2 + np.diff(data['y_position'])**2))
        max_velocity = data['velocity'].max()
        avg_velocity = data['velocity'].mean()
        total_time = data['time_elapsed'].max()
        
        stats_text = f"""
        Performance Statistics:
        • Total Distance: {total_distance:.2f} m
        • Maximum Velocity: {max_velocity:.2f} m/s
        • Average Velocity: {avg_velocity:.2f} m/s
        • Total Time: {total_time:.1f} s
        • Data Points: {len(data)} samples
        """
        
        axes[1,1].text(0.1, 0.9, stats_text, transform=axes[1,1].transAxes, 
                      verticalalignment='top', fontsize=10, fontfamily='monospace')
        axes[1,1].set_xlim(0, 1)
        axes[1,1].set_ylim(0, 1)
        axes[1,1].set_title('Performance Metrics')
        axes[1,1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return {
            'total_distance': total_distance,
            'max_velocity': max_velocity,
            'avg_velocity': avg_velocity,
            'total_time': total_time
        }
    
    def create_mission_planner(self):
        """Create mission planning interface"""
        planner_html = """
        <div style="border: 2px solid #2196F3; padding: 20px; border-radius: 10px;">
            <h3>Mission Planner</h3>
            <div style="display: flex; gap: 20px;">
                <div style="flex: 1;">
                    <h4>Waypoints</h4>
                    <div id="waypoint-list">
                        <!-- Waypoints will be added here -->
                    </div>
                    <div>
                        <input type="number" id="waypoint-x" placeholder="X coordinate" step="0.1" style="width: 80px;">
                        <input type="number" id="waypoint-y" placeholder="Y coordinate" step="0.1" style="width: 80px;">
                        <button onclick="addWaypoint()">Add Waypoint</button>
                    </div>
                </div>
                
                <div style="flex: 1;">
                    <h4>Mission Control</h4>
                    <button onclick="startMission()" style="padding: 10px; background: green; color: white;">Start Mission</button>
                    <button onclick="pauseMission()" style="padding: 10px; background: orange; color: white;">Pause</button>
                    <button onclick="stopMission()" style="padding: 10px; background: red; color: white;">Stop</button>
                    <button onclick="clearWaypoints()" style="padding: 10px;">Clear All</button>
                    
                    <div style="margin-top: 10px;">
                        Mission Status: <span id="mission-status">Ready</span><br>
                        Current Waypoint: <span id="current-waypoint">0</span> of <span id="total-waypoints">0</span>
                    </div>
                </div>
            </div>
        </div>
        
        <script>
        var waypoints = [];
        var mission_active = false;
        var current_waypoint_index = 0;
        
        function addWaypoint() {
            var x = parseFloat(document.getElementById('waypoint-x').value);
            var y = parseFloat(document.getElementById('waypoint-y').value);
            
            if (isNaN(x) || isNaN(y)) {
                alert('Please enter valid coordinates');
                return;
            }
            
            waypoints.push({x: x, y: y});
            updateWaypointDisplay();
            
            document.getElementById('waypoint-x').value = '';
            document.getElementById('waypoint-y').value = '';
        }
        
        function updateWaypointDisplay() {
            var listDiv = document.getElementById('waypoint-list');
            listDiv.innerHTML = '';
            
            waypoints.forEach(function(wp, index) {
                var wpDiv = document.createElement('div');
                wpDiv.innerHTML = `Waypoint ${index + 1}: (${wp.x}, ${wp.y}) <button onclick="removeWaypoint(${index})">Remove</button>`;
                wpDiv.style.margin = '5px 0';
                wpDiv.style.padding = '5px';
                wpDiv.style.background = '#f0f0f0';
                listDiv.appendChild(wpDiv);
            });
            
            document.getElementById('total-waypoints').textContent = waypoints.length;
        }
        
        function removeWaypoint(index) {
            waypoints.splice(index, 1);
            updateWaypointDisplay();
        }
        
        function clearWaypoints() {
            waypoints = [];
            updateWaypointDisplay();
        }
        
        function startMission() {
            if (waypoints.length === 0) {
                alert('Please add waypoints first');
                return;
            }
            
            mission_active = true;
            current_waypoint_index = 0;
            document.getElementById('mission-status').textContent = 'Running';
            document.getElementById('current-waypoint').textContent = '1';
            
            // Execute mission (simplified - would need proper navigation in real implementation)
            console.log('Mission started with', waypoints.length, 'waypoints');
        }
        </script>
        """
        display(HTML(planner_html))

# Usage example in Jupyter notebook:
# robot = RobotWebInterface('192.168.1.100')  # Replace with your Pi IP
# robot.create_control_interface()
# 
# # Load and analyze data
# data = robot.load_data_from_csv('robot_data.csv')
# if data is not None:
#     stats = robot.analyze_robot_performance(data)
#     print(f"Analysis complete: {stats}")
# 
# # Create mission planner
# robot.create_mission_planner()
